# Submit

In [1]:
import pickle
import warnings
import pandas as pd
import unicodedata
import re
warnings.filterwarnings('ignore')

In [2]:
#nlp = spacy.load('es_core_news_sm')
#nlp = spacy.load('pt_core_news_sm')

In [3]:
# Para cargar el modelo
spanish_clf_loaded_model = pickle.load(open('models/LinearSVC_tfidf_SMOTE_spanish_clf.sav', 'rb'))

In [4]:
# Para cargar el modelo
portuguese_clf_loaded_model = pickle.load(open('models/LinearSVC_tfidf_SMOTE_portuguese_clf.sav', 'rb'))

In [5]:
def clean_text(text) :
    text=unicodedata.normalize('NFKD', str(text)).encode('ascii', errors='ignore').decode('utf-8')\
    .lower().replace(r'\\n','').replace(r'-',' ').replace(r'.','').strip()
    text = re.sub("[^a-zA-Z]", " ", str(text))
    return " ".join(text.split())

In [6]:
df_test=pd.read_csv('data/test.csv')

In [7]:
df_test.head()

,id,title,language
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese


In [8]:
df_test.shape

(246955, 3)

In [9]:
print(df_test['language'].value_counts(normalize=True))
print('---------------')
print(df_test['language'].value_counts())

spanish       0.506112
portuguese    0.493888
Name: language, dtype: float64
---------------
spanish       124987
portuguese    121968
Name: language, dtype: int64


In [10]:
df_pred_spanish=df_test[df_test['language']=='spanish']
df_pred_spanish['title']=df_pred_spanish['title'].apply(clean_text)

In [11]:
submit_spanish=spanish_clf_loaded_model.predict(df_pred_spanish['title'])

In [12]:
df_pred_spanish['category']=submit_spanish

In [13]:
df_pred_portuguese=df_test[df_test['language']=='portuguese']
df_pred_portuguese['title']=df_pred_portuguese['title'].apply(clean_text)

In [14]:
submit_portuguese=portuguese_clf_loaded_model.predict(df_pred_portuguese['title'])

In [15]:
df_pred_portuguese['category']=submit_portuguese

In [16]:
df_submit = pd.concat([df_pred_spanish, df_pred_portuguese])

In [17]:
df_submit.sort_values('id',inplace=True)

In [18]:
df_submit.head()

,id,title,language,category
0,0,kit maternidade bolsa mala baby bebe vinho men...,portuguese,DIAPER_BAGS
1,1,trocador de fraldas fisher price feminino rosa...,portuguese,BABY_CHANGING_PADS
2,2,motor ventoinha fiat idea palio a k,portuguese,ENGINE_COOLING_FAN_MOTORS
3,3,amortecedor mola batente d dir new civic k,portuguese,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,cadeirinha de carro bebe princesa princess a kgs,portuguese,BABY_CAR_SEATS


In [19]:
df_submit[['id','title','language','category']].head(10)

,id,title,language,category
0,0,kit maternidade bolsa mala baby bebe vinho men...,portuguese,DIAPER_BAGS
1,1,trocador de fraldas fisher price feminino rosa...,portuguese,BABY_CHANGING_PADS
2,2,motor ventoinha fiat idea palio a k,portuguese,ENGINE_COOLING_FAN_MOTORS
3,3,amortecedor mola batente d dir new civic k,portuguese,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,cadeirinha de carro bebe princesa princess a kgs,portuguese,BABY_CAR_SEATS
5,5,cabo freio mao tras direito vw up cod s e j,portuguese,HAND_BRAKE_CABLES
6,6,mini pc dell optiplex fx atom gb ram ssd gb wifi,portuguese,MINI_PCS
7,7,kit bi xenon lampada h k,portuguese,XENON_KITS
8,8,protetor pe botinha kickboxing karate taekwond...,portuguese,MARTIAL_ARTS_FOOT_GUARDS
9,9,disco rigido externo western digital elements ...,spanish,HARD_DRIVES_AND_SSDS


In [20]:
df_submit[['id','title','language','category']].tail(10)

,id,title,language,category
246945,246945,aplique acab soleira esq outlander r,portuguese,AUTOMOTIVE_BUMPER_GRILLES
246946,246946,set x repuesto para canasta de inodoro liquida cc,spanish,TOILET_BRUSHES
246947,246947,chaleco c patente digitos y calcos p casco mot...,spanish,REFLECTIVE_VESTS
246948,246948,farol mascara negra astra foco duplo,portuguese,AUTOMOTIVE_HEADLIGHTS
246949,246949,polea de goma expansible aluminio p amoladora ...,spanish,INDUSTRIAL_PULLEYS
246950,246950,disco freno delantero ford escort nuevo,spanish,VEHICLE_BRAKE_DISCS
246951,246951,radio comunicador walk talk baofeng s profissi...,portuguese,WALKIE_TALKIES
246952,246952,calculadora de escritorio grande,spanish,CALCULATORS
246953,246953,conj mesa p sala de jantar c cadeiras amsterda...,portuguese,DINING_SETS
246954,246954,cesto residuos tacho basura automatico l l cuotas,spanish,WASTE_BASKETS


In [21]:
df_submit[['id','category']].to_csv('submit/submit_9.csv',index=False)